In [1]:
# Initialize packages
import numpy as np
import pandas as pd
import altair as alt
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer

### Data

In [3]:
train_df = pd.read_csv("../Data/Processed/save_the_earth_train_data.csv", index_col=0)
test_df = pd.read_csv("../Data/Processed/save_the_earth_test_data.csv", index_col=0)

In [4]:
train_df.shape

(2676, 11)

In [5]:
test_df.shape

(669, 11)

In [6]:
train_df.describe(include='all')

,country,year,co2_e,coal_c,elec_g,elec_c,hydro_g,nuclear_g,gas_g,oil_c,oil_g
count,2676,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000,2676.000000
unique,78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,1992.194694,8.407374,0.529499,3855.268834,4655.488341,0.118633,0.043670,1.126443,1.434458,4.130119
std,NaN,13.526486,8.757477,0.815835,5660.509816,5257.367014,0.347776,0.108274,4.095097,1.428530,16.438382
min,NaN,1965.000000,0.052700,0.000000,0.000000,10.200000,0.000000,0.000000,0.000000,0.010300,0.000000
25%,NaN,1981.000000,3.270000,0.018375,0.000000,1220.000000,0.000600,0.000000,0.000000,0.460500,0.000000
50%,NaN,1993.000000,6.730000,0.212500,1950.000000,3210.000000,0.014800,0.000000,0.000000,1.100000,0.000000
75%,NaN,2004.000000,10.500000,0.741500,5615.000000,6062.500000,0.065625,0.006850,0.485500,1.970000,1.292500


In [7]:
X_train = train_df.drop(columns=["co2_e"])
X_test = test_df.drop(columns=["co2_e"])
y_train = train_df["co2_e"]
y_test = test_df["co2_e"]

In [8]:
y_test.shape

(669,)

### Preprocessing

Based on the nature of the data and the EDA results, the following assumption and preprocessing would be made
- A **naive assumption** that there is no temporal dependency between observations (i.e. observations among years) is made. `year` would be removed to prevent the model from exploiting the temporal feature for future-looking. Temporal feature treatment, e.g. time series split and time series cross-validation, could be considered later
- Scaling will be applied to all numeric features to standardize them to a common scale.
- OneHotEncoding will be applied to the categorical feature `country`.

In [9]:
# Lists of feature names
drop_feats = ['year']
categorical_feats = ['country']
numerical_feats = ['coal_c', 'elec_g', 'elec_c', 'hydro_g', 'nuclear_g', 'gas_g', 'oil_c', 'oil_g']

# Create the column transformer
preprocessor = make_column_transformer(
    ('drop', drop_feats),
    (StandardScaler(), numerical_feats),
    (OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype='int'), categorical_feats)
)

preprocessor.verbose_feature_names_out = False

preprocessor

ColumnTransformer(transformers=[('drop', 'drop', ['year']),
                                ('standardscaler', StandardScaler(),
                                 ['coal_c', 'elec_g', 'elec_c', 'hydro_g',
                                  'nuclear_g', 'gas_g', 'oil_c', 'oil_g']),
                                ('onehotencoder',
                                 OneHotEncoder(dtype='int',
                                               handle_unknown='ignore',
                                               sparse_output=False),
                                 ['country'])],
                  verbose_feature_names_out=False)

### Model Training

We used various regression models with R^2 as the scoring metrics and carry out 10-fold cross-validation with each model to find the best performing models. Based on the validation results, the model using k-nearest neighbors (k-nn) 
algorithm is the best performing model with R^2 of 0.949.

In [10]:
models = {
    "Baseline": DummyRegressor(),
    "KNN_reg": KNeighborsRegressor(),
    "Ridge": Ridge(),
    "SVR": SVR(),
}
score_types = {
    "r2": "r2",
}

In [11]:
cross_val_results = dict()

for name, model in models.items():
    pipe = make_pipeline(preprocessor, model)
    cross_val_results[name] = (
        pd.DataFrame(
            cross_validate(
                pipe,
                X_train,
                y_train,
                cv=10,
                scoring=score_types,
                return_train_score=True,
            )
        )
        .agg(["mean", "std"])
        .round(3)
        .T
    )

cross_val_results_df = pd.concat(
    cross_val_results,
    axis="columns"
)
cross_val_results_df

Baseline        KNN_reg         Ridge           SVR       
               mean    std    mean    std   mean    std   mean    std
fit_time      0.002  0.000   0.003  0.001  0.006  0.003  0.212  0.026
score_time    0.001  0.000   0.011  0.016  0.002  0.001  0.045  0.001
test_r2      -0.003  0.004   0.953  0.022  0.915  0.021  0.714  0.057
train_r2      0.000  0.000   0.975  0.003  0.926  0.002  0.726  0.006

### Hyperparameter Optimization

The hyperparameter `n_neighbors` and `max_categories` was chosen using 10-fold cross validation with R^2 as the classification metric to improve the model performance. Based on the validation results, the KNN model has achieved a R^2(`mean_test_r2`) of 0.975.

In [12]:
param_dist = {
    "kneighborsregressor__n_neighbors": randint(1, 20),
    "columntransformer__onehotencoder__max_categories": randint(1, X_train['country'].unique().shape[0])
}

pipe_best_model = make_pipeline(preprocessor, KNeighborsRegressor())

random_search = RandomizedSearchCV(
    pipe_best_model,
    param_distributions=param_dist,
    cv=10,
    n_iter=200,
    scoring=score_types,
    n_jobs=-1,
    refit="r2",
    return_train_score=True,
)
random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=10,
                   estimator=Pipeline(steps=[('columntransformer',
                                              ColumnTransformer(transformers=[('drop',
                                                                               'drop',
                                                                               ['year']),
                                                                              ('standardscaler',
                                                                               StandardScaler(),
                                                                               ['coal_c',
                                                                                'elec_g',
                                                                                'elec_c',
                                                                                'hydro_g',
                                                                                'nuclear_g',
                                                                                'gas_g',
                                                                                'oil_c',
                                                                                'oil_g']),
                                                                              ('onehotencoder',
                                                                               OneHotEncoder(dtype='int',
                                                                                             handle_unknown='ignore',
                                                                                             sparse_output=False),
                                                                               ['country'])],
                                                                verbose_...
                                              KNeighborsRegressor())]),
                   n_iter=200, n_jobs=-1,
                   param_distributions={'columntransformer__onehotencoder__max_categories': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x123c38690>,
                                        'kneighborsregressor__n_neighbors': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x15336fb10>},
                   refit='r2', return_train_score=True, scoring={'r2': 'r2'})

In [13]:
pd.DataFrame(random_search.cv_results_)[['param_columntransformer__onehotencoder__max_categories', 'param_kneighborsregressor__n_neighbors', 'mean_test_r2', 'std_test_r2']].sort_values('mean_test_r2', ascending=False)[:20]

,param_columntransformer__onehotencoder__max_categories,param_kneighborsregressor__n_neighbors,mean_test_r2,std_test_r2
86,26,1,0.975372,0.013478
30,1,1,0.975200,0.013607
191,2,1,0.975200,0.013607
44,12,1,0.975089,0.013530
59,10,1,0.975089,0.013530
131,43,1,0.974802,0.013708
78,35,1,0.974728,0.013615
186,33,1,0.974557,0.013565
130,31,1,0.974541,0.013581
5,58,1,0.974470,0.013930


In [14]:
# Scaled data export
scaled_X_train = random_search.best_estimator_.named_steps['columntransformer'].transform(X_train)
scaled_X_test = random_search.best_estimator_.named_steps['columntransformer'].transform(X_test)

scaled_X_train = pd.DataFrame(scaled_X_train, columns=random_search.best_estimator_.named_steps['columntransformer'].get_feature_names_out().tolist(), index=X_train.index)
scaled_X_test = pd.DataFrame(scaled_X_test, columns=random_search.best_estimator_.named_steps['columntransformer'].get_feature_names_out().tolist(), index=X_test.index)

scaled_X_train.to_csv("../Data/Processed/scaled_save_the_earth_train_data.csv", index=True)
scaled_X_test.to_csv("../data/processed/scaled_save_the_earth_test_data.csv", index=True)

### Test Results

Our prediction model performed quite well on test data, with a final overall R^2 of 0.976, which is promising for predicting a country's CO2 emission per capita given the energy generation and consumption data.

In [15]:
random_search.score(X_test, y_test)

0.975645926748788

### Improvement

To further improve this model in future with hopes of arriving one that could be used, there are several improvements we can suggest for later revision.
- As mentioned in Preprocessing, there could possibly be temporal dependency between observations and temporal treatments could be considered.
- In the EDA above, we discovered there are collinearity between `oil_c` and `elec_c`, `oil_g` and `gas_g`. Though it might not affect the predictive power of models, it harms the interpretation of the coefficients of linear models. Collinearity reduction treatment e.g. feature removal, dimension reduction technique, etc., could be considered.
- Assumed that co2_emission might be still in increasing trend in the future, KNN may not predict well beyond the range of values input in your training data. Other models with similar predictive power which can predict out-of-range input data could be considered.